# Prune leaf ROIs in an annotation ontology file without voxel-counts
This notebook deletes nodes in an anatomical ontology (AO) text file if these are leaf nodes without voxel counts (VC) in annotation volume (AV).

- input
    - 1_VC.json: AO file with VC
- output
    - 1_VC_pruned.json: pruned AO file with VC

# Set variables

In [1]:
dir_data = 'data'
fn_input_AO = '1_VC.json' # filename for anatomical ontology with voxel counts
fn_output_AO = '1_VC_pruned.json'

In [2]:
import os
import json
import copy
import numpy as np
from collections import OrderedDict
from jsonpath_rw import jsonpath, parse

# Load data

In [4]:
with open(os.path.join(dir_data, fn_input_AO)) as f:
    df_VC = json.load(f, object_pairs_hook=OrderedDict)

# Delete ROIs in an annotation ontology file if it's a leaf without voxel counts

In [5]:
def Delete_leaf_node_WO_VC(match_id, match_fullpath):
    global df_VC_pruned
    global leaf_count
    bool1 = eval(("df_VC_pruned['msg'][0]"+\
                  str(match_fullpath).replace('.','')\
                  .replace('children', "['children']")\
                  .replace('id','')+"['children'] == []"))
    bool2 = eval(("df_VC_pruned['msg'][0]"+\
                  str(match_fullpath).replace('.','')\
                  .replace('children', "['children']")\
                  .replace('id','')+"['voxel_count'] is None"))
    if bool1:
        leaf_count = leaf_count + 1
    if bool1 and bool2:
        print(("***** deleted ID: " + \
              str(eval("df_VC_pruned['msg'][0]"+\
                       str(match_fullpath).replace('.','')\
                       .replace('children', "['children']")\
                       .replace('id', "['id']")))))
        exec("del df_VC_pruned['msg'][0]"+\
             str(match_fullpath).replace('.','')\
             .replace('children', "['children']")\
             .replace('id', ''))
        return match_id * (-1) # deleted ids are assigned as negative
    else:
        print(("remained ID: " + 
               str(eval("df_VC_pruned['msg'][0]"+\
                        str(match_fullpath)\
                        .replace('.','')\
                        .replace('children', "['children']")\
                        .replace('id', "['id']")))))
        return match_id # remained ids in positive

In [6]:
leaf_count = 0
jsonpath_expr = parse('$..id')
df_VC_pruned = copy.deepcopy(df_VC)
temp_list = []
temp_list = [Delete_leaf_node_WO_VC(match.value, match.full_path)\
            for match in reversed(jsonpath_expr.find(df_VC_pruned['msg'][0]))]
DeletedLeafs = np.array(temp_list)

***** deleted ID: 304325711
***** deleted ID: 624
***** deleted ID: 65
***** deleted ID: 57
***** deleted ID: 49
***** deleted ID: 43
***** deleted ID: 34
***** deleted ID: 25
***** deleted ID: 18
***** deleted ID: 11
***** deleted ID: 3
***** deleted ID: 1119
***** deleted ID: 1112
***** deleted ID: 1103
***** deleted ID: 1095
***** deleted ID: 1087
***** deleted ID: 1040
***** deleted ID: 1078
***** deleted ID: 1071
***** deleted ID: 1063
***** deleted ID: 1055
***** deleted ID: 1032
***** deleted ID: 1024
***** deleted ID: 164
remained ID: 153
remained ID: 145
remained ID: 140
remained ID: 129
***** deleted ID: 124
***** deleted ID: 116
remained ID: 108
remained ID: 98
***** deleted ID: 89
remained ID: 81
remained ID: 73
***** deleted ID: 563
***** deleted ID: 547
***** deleted ID: 70
***** deleted ID: 730
remained ID: 611
remained ID: 595
remained ID: 802
remained ID: 1083
***** deleted ID: 722
***** deleted ID: 1068
remained ID: 673
remained ID: 681
remained ID: 690
remained ID: 7

remained ID: 312782554
remained ID: 312782550
remained ID: 312782546
***** deleted ID: 340
***** deleted ID: 308
***** deleted ID: 683
***** deleted ID: 635
***** deleted ID: 241
***** deleted ID: 532
remained ID: 22
remained ID: 622
remained ID: 590
remained ID: 687
remained ID: 430
***** deleted ID: 606
remained ID: 542
remained ID: 886
remained ID: 330
remained ID: 274
remained ID: 610
***** deleted ID: 545
remained ID: 434
remained ID: 442
remained ID: 879
***** deleted ID: 480149338
***** deleted ID: 480149334
***** deleted ID: 480149330
***** deleted ID: 480149326
***** deleted ID: 480149322
***** deleted ID: 480149318
***** deleted ID: 480149314
***** deleted ID: 480149310
***** deleted ID: 480149306
***** deleted ID: 480149302
***** deleted ID: 480149298
***** deleted ID: 480149294
***** deleted ID: 480149290
***** deleted ID: 480149286
***** deleted ID: 480149282
***** deleted ID: 480149278
***** deleted ID: 480149274
***** deleted ID: 480149270
***** deleted ID: 480149266
***

# Save a pruned annotation ontology file with voxel counts 

In [20]:
with open(os.path.join(dir_data, fn_output_AO), mode='w') as fw:
    json.dump(df_VC_pruned, fw, indent=4)

# Check data

In [7]:
print('# all IDs in AO: '+str(len(DeletedLeafs))) # 1327
print('# remained IDs: '+str(sum(DeletedLeafs>0))) # 837
print('# deleted IDs: '+str(sum(DeletedLeafs<0))) # 490
print('# leaf count: ' +str(leaf_count))

# all IDs in AO: 1327
# remained IDs: 837
# deleted IDs: 490
# leaf count: 1130
